# **Implementing RAG pipeline for querying and extracting relevant information from list of PDF docs**

Due to security/privacy concern over using Chatmodels on sensitive datas, It is not recommended to train the model using private sensitive datas.

Therefore, RAG pipeline is used for extracting relevant infos from sensitive datas, without requiring to train the model again.



In [ ]:
# We will be using langchain framework to build end-to-end pipeline for RAG flow.
# langchain is most popular framework to building LLM usecase pipelines.

!pip install -r requirements.txt

In [9]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import chains, vectorstores
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
import os
import PyPDF2
import pypdf
import re
from langchain.embeddings import HuggingFaceEmbeddings

# Indexing the Documents and storing it into **VectorStores** Databases.

These documents will be converted into fixed sized chunks and then stored in document embedded forms.

This way, it will be easy to retrieve the required documents.

In [10]:
pdf_document_load = PyPDF2.PdfReader(open("/content/sell_my_dream.pdf", "rb"))

In [11]:
len(pdf_document_load.pages)

4

In [12]:
# Start building pipelines be intiating the model

# load enviroment variables and API_KEYs
load_dotenv()

# Instantiate chat model
def init_chatmodel():
  chatmodel = ChatOpenAI(model="gpt-3.5-turbo")
  return chatmodel

# instantiate Embeddings
def init_embeddings():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
  return embeddings

# load the pdf documents and load its contents for further processing
def load_pdf_document(pdfPath):
  pdf_document = PyPDFLoader(pdfPath)
  documents = pdf_document.load()
  return documents

# split the documents into multiple chunks(of size 1000 tokens) for indexing
def split_pdf_documents(documents):
  doc_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  docs = doc_splitter.split_documents(documents)
  return docs

In [13]:
# now, document embedding is to be done using BERT based OpenAIEmbeddings
"""
This method leverages the power of a OpenAIEmbeddings model for embedding generation
and the efficiency of Chroma for storing and querying embeddings, facilitating
various downstream tasks like document retrieval and clustering.
ChromaDB then adds each document's embedding to the collection using unique
document identifiers
"""

def create_vectorDB_store(textdocs, embeddingFun):
  vectorDB = vectorstores.Chroma.from_documents(textdocs, embeddingFun)
  return vectorDB


In [14]:
# Retrieval Part
"""
Given a query and documents, this method will fetch the relevant documents
based on similarity search.
"""

# It can produce n number of relevant documents chunks requested by user.

def get_retrieved_docs(n_documents, stored_VectorDB):
  retrieved_docs = stored_VectorDB.as_retriever(search_type="similarity", search_kwargs={"k":n_documents})
  return retrieved_docs

# Combining all components to create the RAG pipeline

In [47]:
# Integrate all components into RAG pipeline for end-to-end chain flow

loaded_docs = load_pdf_document("/content/sell_my_dream.pdf")
textdocs = split_pdf_documents(loaded_docs)
print("chunk size: ", len(textdocs))
vectordb = create_vectorDB_store(textdocs, embeddingFun=init_embeddings())
retrievers = get_retrieved_docs(3, vectordb)


chunk size:  4


In [16]:
# get relevant docs as per query
query = "Why does the author compare Neruda to a Renaissance pope?"
relevant_docs = retrievers.invoke(query)

In [18]:
# print those relevant docs
for i, doc in enumerate(relevant_docs, 1):
  print(f"Document {i}: \n {doc.metadata} ")
  print("\n-------------------------------------------------------\n")

Document 1: 
 {'language': 'en-US', 'source': 'https://www.apple.com/', 'title': 'Apple'} 

-------------------------------------------------------

Document 2: 
 {'language': 'en-US', 'source': 'https://www.apple.com/', 'title': 'Apple'} 

-------------------------------------------------------

Document 3: 
 {'language': 'en-US', 'source': 'https://www.apple.com/', 'title': 'Apple'} 

-------------------------------------------------------



In [ ]:
# Combined chain with LLM model in it.
chatmodel = init_chatmodel()

rag_chain = chains.RetrievalQA.from_chain_type(
    llm=chatmodel,
    chain_type="stuff",
    retriever=retrievers,
    return_source_documents=True
)

In [ ]:
# Calling RAG pipeline with query:
query_result1 = rag_chain.invoke({"query": "Why does the author compare Neruda to a Renaissance pope?"})

print("answer: ", query_result1["result"])

In [ ]:
# We can also know which documents consitutes the answer:
for document in query_result1["source_documents"]:
  print(re.sub(r"\s+", " ", document.page_content.strip()))
  print("\n-------------------------------------------------------\n")

In [ ]:
# Setting up memory context for Chat of previous query.
chat_history1 = [(query_result1["query"], query_result1["result"])]
query_result2 = rag_chain.invoke({
    "query": "Are there any other reasons of comparing Neruda to a Renaissance pope?",
    "chat_history": chat_history1
})
print("answer: ", query_result2["result"])

In [ ]:
for document in query_result2["source_documents"]:
  print(re.sub(r"\s+", " ", document.page_content.strip()))
  print("\n-------------------------------------------------------\n")

# WebScrapping using RAG based pipeline



In [15]:
# define weburl for webscrapping

web_url = ["https://www.apple.com/"]
web_content_loader = WebBaseLoader(web_url)
loaded_docs = web_content_loader.load()
textdocs = split_pdf_documents(loaded_docs)
print("chunk size: ", len(textdocs))
vectordb = create_vectorDB_store(textdocs, embeddingFun=init_embeddings())
retrievers = get_retrieved_docs(3, vectordb)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


chunk size:  6


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
# get relevant docs as per query
web_query = "What are the new products announced at apple event?"
relevant_web_docs = retrievers.invoke(web_query)

In [20]:
# print those relevant docs
for i, doc in enumerate(relevant_web_docs, 1):
  print(f"Document {i}: \n {doc} ")
  print("\n-------------------------------------------------------\n")

Document 1: 
 page_content='Apple


Apple

AppleStoreMaciPadiPhoneWatchVisionAirPodsTV & HomeEntertainmentAccessoriesSupport


0+

									For a limited time, shop your state's eligible products tax‑free — online and in‑store. Learn more


 

 

Buy Mac or iPad for college


with education savings

Get a gift card up to $150*


Only at the Apple Store


Shop


Buy Mac or iPad for college


with education savings

Get AirPods with Mac*


Apple Pencil with iPad*


Only at the Apple Store


Shop


 


 

MacBook Air
Lean. Mean. M3 machine.

Learn more
Buy


 

iPhone
Our most powerful cameras yet. Ultrafast chips. And USB-C.

Learn more
Shop iPhone

 

iPad Pro
Unbelievably thin. Incredibly powerful.

Learn more
Buy


 

Apple Intelligence
AI for the rest of us.
Coming in beta this fall1

Learn more
Watch the film


 

Apple Vision Pro
The era of spatial computing is here.

Learn more
Buy


 

Apple at Work
Work. Like never before.

Learn more


 

Apple Card
Get up to 3% Daily Cash back